# **Trắc nghiệm**

1B 2B 3B 4B 5B 6B 7A 8B 9A 10B

# **Tự luận**

Khai báo thư viện

In [ ]:
import pandas as pd

**Tải file dữ liệu**

In [ ]:
from google.colab import files
files.upload()    # car-sales-data.csv

Saving car-sales-data.csv to car-sales-data.csv


{'car-sales-data.csv': b'Make,Colour,Odometer,Doors,Price\r\nToyota,White,150043,4,"$4,000"\r\nHonda,Red,87899,4,"$5,000"\r\nToyota,Blue,,3,"$7,000"\r\nBMW,Black,11179,5,"$22,000"\r\nNissan,White,213095,4,"$3,500"\r\nToyota,Green,,4,"$4,500"\r\nHonda,,,4,"$7,500"\r\nHonda,Blue,,4,\r\nToyota,White,60000,,\r\n,White,31600,4,"$9,700"'}

**Đọc file dữ liệu**

In [ ]:
df = pd.read_csv('car-sales-data.csv', index_col=[0])
df

,Colour,Odometer,Doors,Price
Make,,,,
Toyota,White,150043.0,4.0,"$4,000"
Honda,Red,87899.0,4.0,"$5,000"
Toyota,Blue,NaN,3.0,"$7,000"
BMW,Black,11179.0,5.0,"$22,000"
Nissan,White,213095.0,4.0,"$3,500"
Toyota,Green,NaN,4.0,"$4,500"
Honda,NaN,NaN,4.0,"$7,500"
Honda,Blue,NaN,4.0,NaN
Toyota,White,60000.0,NaN,NaN


**Observe dataframe - Quan sát dữ liệu**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, Toyota to nan
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Colour    9 non-null      object 
 1   Odometer  6 non-null      float64
 2   Doors     9 non-null      float64
 3   Price     8 non-null      object 
dtypes: float64(2), object(2)
memory usage: 400.0+ bytes


# Handle missing values - Xử lý các giá trị bị thiếu (NaN)

In [ ]:
# Odometer là số km đã đi
# Fill các giá trị Odometer bị thiếu với giá trị trung bình (không được xóa hoặc fill=0)
df["Odometer"].fillna(df["Odometer"].mean(), inplace=True)
df

<ipython-input-5-27b3205f1d24>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Odometer"].fillna(df["Odometer"].mean(), inplace=True)


,Colour,Odometer,Doors,Price
Make,,,,
Toyota,White,150043.000000,4.0,"$4,000"
Honda,Red,87899.000000,4.0,"$5,000"
Toyota,Blue,92302.666667,3.0,"$7,000"
BMW,Black,11179.000000,5.0,"$22,000"
Nissan,White,213095.000000,4.0,"$3,500"
Toyota,Green,92302.666667,4.0,"$4,500"
Honda,NaN,92302.666667,4.0,"$7,500"
Honda,Blue,92302.666667,4.0,NaN
Toyota,White,60000.000000,NaN,NaN


In [ ]:
# Loại bỏ những xe thiếu giá bán
df.dropna(subset=['Price'], inplace=True)
df

,Colour,Odometer,Doors,Price
Make,,,,
Toyota,White,150043.000000,4.0,"$4,000"
Honda,Red,87899.000000,4.0,"$5,000"
Toyota,Blue,92302.666667,3.0,"$7,000"
BMW,Black,11179.000000,5.0,"$22,000"
Nissan,White,213095.000000,4.0,"$3,500"
Toyota,Green,92302.666667,4.0,"$4,500"
Honda,NaN,92302.666667,4.0,"$7,500"
NaN,White,31600.000000,4.0,"$9,700"


# Most expensive car - Xe đắt nhất

Vì cột Price hiện tại đang là kiểu object (string) nên cần covert kiểu dữ liệu trước khi có thể so sánh

In [ ]:
# Xây dựng hàm để loại bỏ dấu "$" và "," trong cột Price
def convert_price(price):
    price = price.replace("$", "")
    price = price.replace(",", "")
    return price

# Chuyển đổi kiểu dữ liệu cột Price từ object sang float (tạo cột mới)
df["Price $"] = df["Price"].map(convert_price).astype(float)
df

,Colour,Odometer,Doors,Price,Price $
Make,,,,,
Toyota,White,150043.000000,4.0,"$4,000",4000.0
Honda,Red,87899.000000,4.0,"$5,000",5000.0
Toyota,Blue,92302.666667,3.0,"$7,000",7000.0
BMW,Black,11179.000000,5.0,"$22,000",22000.0
Nissan,White,213095.000000,4.0,"$3,500",3500.0
Toyota,Green,92302.666667,4.0,"$4,500",4500.0
Honda,NaN,92302.666667,4.0,"$7,500",7500.0
NaN,White,31600.000000,4.0,"$9,700",9700.0


Tìm ra chiếc xe đắt nhất

In [ ]:
# Tìm giá trị lớn nhất của cột Price bằng hàm max()
max_price = df["Price $"].max()
# Hiện ra dòng có giá trị ở cột Price = giá trị max
df[df["Price $"] == max_price]

,Colour,Odometer,Doors,Price,Price $
Make,,,,,
BMW,Black,11179.0,5.0,"$22,000",22000.0


# Brand with lowest Price per KM - Thương hiệu có giá bán rẻ nhất tính trên mỗi KM

Tạo ra 1 cột mới chứa thông tin giá bán trên mỗi KM

In [ ]:
# Giá bán trên mỗi KM = Giá bán / số KM đã đi
df["Price per KM"] = df["Price $"]/df["Odometer"]
df

,Colour,Odometer,Doors,Price,Price $,Price per KM
Make,,,,,,
Toyota,White,150043.000000,4.0,"$4,000",4000.0,0.026659
Honda,Red,87899.000000,4.0,"$5,000",5000.0,0.056883
Toyota,Blue,92302.666667,3.0,"$7,000",7000.0,0.075837
BMW,Black,11179.000000,5.0,"$22,000",22000.0,1.967976
Nissan,White,213095.000000,4.0,"$3,500",3500.0,0.016425
Toyota,Green,92302.666667,4.0,"$4,500",4500.0,0.048753
Honda,NaN,92302.666667,4.0,"$7,500",7500.0,0.081254
NaN,White,31600.000000,4.0,"$9,700",9700.0,0.306962


Tìm thương hiệu có giá bán rẻ nhất trên mỗi KM

In [ ]:
# Tìm giá trị nhỏ nhất của cột giá bán trên mỗi KM vừa tạo bằng hàm min()
min_ppk = df["Price per KM"].min()

# Hiện ra thương hiệu có giá trị cột "Price per KM" = giá trị min vừa tìm
    # Bước này khác với bước tìm xe ở trên, thay vì hiện ra cả dòng thì mình chỉ hiện tên thương hiệu (cột Make và cột 'Price per KM')
df[df["Price per KM"] == min_ppk]['Price per KM']

,Price per KM
Make,
Nissan,0.016425


# Average brand price - Giá trung bình của các hãng xe

In [ ]:
# Giá trị trung bình: mean()
# Tạo ra 1 DataFrame mới bao gồm tên các hãng xe và giá bán trung bình của từng hãng
mean_make = df.groupby("Make").mean("Price $")["Price $"]
mean_make

,Price $
Make,
BMW,22000.000000
Honda,6250.000000
Nissan,3500.000000
Toyota,5166.666667


In [ ]:
# Sắp xếp theo thứu tự giá bán giảm dần
mean_make.sort_values(ascending=False)

,Price $
Make,
BMW,22000.000000
Honda,6250.000000
Toyota,5166.666667
Nissan,3500.000000
